In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
#import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
pip install paddlepaddle

In [ ]:
pip install paddleocr

In [ ]:
pip install matplotlib

In [ ]:
#function Chạy Yolo

In [ ]:
from paddleocr import PaddleOCR

paddle_ocr = None

def read(image):
  global paddle_ocr

  if paddle_ocr is None:
    paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log=False)

  result = paddle_ocr.ocr(image, cls=True)
  text = ''

  for idx in range(len(result)):
    res = result[idx]
    for line in res:
        if len(text) > 0:
            text += '; '
        text += f'{line[1][0]}'

  return {'text': text}

In [ ]:
from paddleocr import PaddleOCR, draw_ocr

# Khởi tạo mô hình PaddleOCR (có thể thay đổi config tùy vào nhu cầu)
ocr = PaddleOCR(use_angle_cls=True, use_gpu=False)

def extract_text(image, bbox):
    x, y, w, h = bbox
    roi = image[y:y+h, x:x+w]

    if 0 in roi.shape:
        return 'no number'
    else:
        # Nhận diện văn bản từ vùng quan trọng (ROI) bằng PaddleOCR
        result = ocr.ocr(roi)

        # Xử lý kết quả nhận diện
        if len(result) > 0:
            text = ' '.join([word[1][0] for word in result[0]])
            return text.strip()
        else:
            return 'no text'


download https://paddleocr.bj.bcebos.com/PP-OCRv4/chinese/ch_PP-OCRv4_det_infer.tar to /root/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer/ch_PP-OCRv4_det_infer.tar


100%|██████████| 4.89M/4.89M [00:06<00:00, 771kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/chinese/ch_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer/ch_PP-OCRv4_rec_infer.tar


100%|██████████| 11.0M/11.0M [00:17<00:00, 625kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:14<00:00, 148kiB/s]

[2023/12/16 13:36:44] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='

In [ ]:
# settings
INPUT_WIDTH =  640
INPUT_HEIGHT = 640
def get_detections(img,net):
    # 1.CONVERT IMAGE TO YOLO FORMAT
    image = img.copy()
    row, col, d = image.shape

    max_rc = max(row,col)
    input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
    input_image[0:row,0:col] = image

    # 2. GET PREDICTION FROM YOLO MODEL
    blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WIDTH,INPUT_HEIGHT),swapRB=True,crop=False)
    net.setInput(blob)
    preds = net.forward()
    detections = preds[0]

    return input_image, detections

def non_maximum_supression(input_image,detections):

    # 3. FILTER DETECTIONS BASED ON CONFIDENCE AND PROBABILIY SCORE

    # center x, center y, w , h, conf, proba
    boxes = []
    confidences = []

    image_w, image_h = input_image.shape[:2]
    x_factor = image_w/INPUT_WIDTH
    y_factor = image_h/INPUT_HEIGHT

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4] # confidence of detecting license plate
        if confidence > 0.4:
            class_score = row[5] # probability score of license plate
            if class_score > 0.25:
                cx, cy , w, h = row[0:4]

                left = int((cx - 0.5*w)*x_factor)
                top = int((cy-0.5*h)*y_factor)
                width = int(w*x_factor)
                height = int(h*y_factor)
                box = np.array([left,top,width,height])

                confidences.append(confidence)
                boxes.append(box)

    # 4.1 CLEAN
    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    # 4.2 NMS
    index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45)

    return boxes_np, confidences_np, index

def drawings(image,boxes_np,confidences_np,index):
    # 5. Drawings
    for ind in index:
        x,y,w,h =  boxes_np[ind]
        bb_conf = confidences_np[ind]
        conf_text = 'plate: {:.0f}%'.format(bb_conf*100)
        #license_text = extract_text(image,boxes_np[ind])
        cropped_img = image[y:y+h, x:x+w]
        license_text = extract_text(image,boxes_np[ind])



        cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.rectangle(image,(x,y-30),(x+w,y),(255,0,255),-1)
        cv2.rectangle(image,(x,y+h),(x+w,y+h+25),(0,0,0),-1)


        cv2.putText(image,conf_text,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),1)
        cv2.putText(image,license_text,(x,y+h+27),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),1)

    return image

In [ ]:
# predictions flow with return result
def yolo_predictions(img,net):
    # step-1: detections
    input_image, detections = get_detections(img,net)
    # step-2: NMS
    boxes_np, confidences_np, index = non_maximum_supression(input_image, detections)
    print(index)
    # step-3: Drawings
    result_img = drawings(img,boxes_np,confidences_np,index)
    return result_img

In [ ]:
def load_yolo_model():
    net = cv2.dnn.readNetFromONNX('/content/drive/MyDrive/ColabNotebooks/DataSet/yolov5/runs/train/Model/weights/best.onnx')
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
def outputText(filename):

    # settings
    INPUT_WIDTH =  640
    INPUT_HEIGHT = 640

    # LOAD THE IMAGE
    img = cv2.imread('/content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/upload/{}'.format(filename))
    # LOAD YOLO MODEL
    #load_yolo_model()
    net = cv2.dnn.readNetFromONNX('/content/drive/MyDrive/ColabNotebooks/DataSet/yolov5/runs/train/Model/weights/best.onnx')
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    # Predict and draw bounding boxes
    result_img = yolo_predictions(img, net)

    input_image, detections = get_detections(img,net)
    boxes_np, confidences_np, index = non_maximum_supression(input_image, detections)

    cv2.imwrite('/content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/predict/{}'.format(filename), cv2.cvtColor(result_img, cv2.COLOR_RGB2BGR))
    for index2 in index:
        i = index2
    return extract_text(img, boxes_np[i])



#if __name__ == "__main__":
    #print(main("N95.jpeg"))



In [ ]:
#chạy app

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://1qer17ya8vc-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
!pip install flask

In [ ]:
cd /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/

/content/drive/MyDrive/ColabNotebooks/DataSet/WebApp


In [ ]:
#chính thức

In [ ]:
from flask import Flask, render_template, request
import os

# Webserver gateway interface
app = Flask(__name__, template_folder = "/content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/template", static_url_path='/content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static')

#/content/drive/MyDrive/static/upload/N98.jpeg
BASE_PATH = os.getcwd()
UPLOAD_PATH = os.path.join(
    BASE_PATH, '/content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/upload')

@app.route('/', methods=['POST', 'GET'])
def index():
    if request.method == 'POST':
        upload_file = request.files['image_name']
        filename = upload_file.filename
        path_save = os.path.join(UPLOAD_PATH, filename)
        upload_file.save(path_save)



        net = cv2.dnn.readNetFromONNX('/content/drive/MyDrive/ColabNotebooks/DataSet/yolov5/runs/train/Model/weights/best.onnx')
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
        text = outputText(filename)
        #text = OCR(path_save, filename)
        return render_template('index.html', upload=True, upload_image=filename, text = text)
        #return render_template('index.html', upload=True, upload_image=filename)

    return render_template('index.html', upload=False)

if __name__ == "__main__":
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:49:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:49:03] "GET /favicon.ico HTTP/1.1" 404 -


[12]
[2023/12/16 15:49:20] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.017711639404296875
[2023/12/16 15:49:20] ppocr DEBUG: cls num  : 2, elapsed : 0.02129054069519043
[2023/12/16 15:49:20] ppocr DEBUG: rec_res num  : 2, elapsed : 0.2520928382873535
[2023/12/16 15:49:21] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.014804840087890625
[2023/12/16 15:49:21] ppocr DEBUG: cls num  : 2, elapsed : 0.020357370376586914
[2023/12/16 15:49:21] ppocr DEBUG: rec_res num  : 2, elapsed : 0.24547767639160156


INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:49:21] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:49:22] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/upload/0c62750f-9a1f-47c2-b7ff-7c76e7dabb9d.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:49:22] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/predict/0c62750f-9a1f-47c2-b7ff-7c76e7dabb9d.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:49:22] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/roi/0c62750f-9a1f-47c2-b7ff-7c76e7dabb9d.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:49:23] "GET /favicon.ico HTTP/1.1" 404 -


[6]
[2023/12/16 15:49:52] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.012973785400390625
[2023/12/16 15:49:52] ppocr DEBUG: cls num  : 2, elapsed : 0.02295827865600586
[2023/12/16 15:49:52] ppocr DEBUG: rec_res num  : 2, elapsed : 0.2601156234741211
[2023/12/16 15:49:53] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.01165461540222168
[2023/12/16 15:49:53] ppocr DEBUG: cls num  : 2, elapsed : 0.02305293083190918
[2023/12/16 15:49:53] ppocr DEBUG: rec_res num  : 2, elapsed : 0.25586962699890137


...

INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:52:03] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:52:03] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/upload/IMG_8854.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:52:03] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/predict/IMG_8854.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:52:03] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/roi/IMG_8854.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:52:05] "GET /favicon.ico HTTP/1.1" 404 -


[3]
[2023/12/16 15:52:52] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.2763185501098633
[2023/12/16 15:52:52] ppocr DEBUG: cls num  : 2, elapsed : 0.01833343505859375
[2023/12/16 15:52:52] ppocr DEBUG: rec_res num  : 2, elapsed : 0.2601945400238037
[2023/12/16 15:52:53] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.2799568176269531
[2023/12/16 15:52:53] ppocr DEBUG: cls num  : 2, elapsed : 0.018890380859375
[2023/12/16 15:52:54] ppocr DEBUG: rec_res num  : 2, elapsed : 0.260455846786499


INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:52:54] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:52:54] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/upload/IMG_8858.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:52:54] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/predict/IMG_8858.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:52:54] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/roi/IMG_8858.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:52:56] "GET /favicon.ico HTTP/1.1" 404 -


[2]
[2023/12/16 15:53:42] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.14611506462097168
[2023/12/16 15:53:42] ppocr DEBUG: cls num  : 2, elapsed : 0.018956422805786133
[2023/12/16 15:53:42] ppocr DEBUG: rec_res num  : 2, elapsed : 0.2558779716491699
[2023/12/16 15:53:43] ppocr DEBUG: dt_boxes num : 5, elapsed : 0.18006062507629395
[2023/12/16 15:53:43] ppocr DEBUG: cls num  : 5, elapsed : 0.03862738609313965
[2023/12/16 15:53:44] ppocr DEBUG: rec_res num  : 5, elapsed : 0.611152172088623


INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:53:44] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:53:44] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/upload/IMG_8856.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:53:44] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/predict/IMG_8856.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:53:44] "GET /content/drive/MyDrive/ColabNotebooks/DataSet/WebApp/static/roi/IMG_8856.jpg HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 15:53:46] "GET /favicon.ico HTTP/1.1" 404 -
